In [1]:
import caffe

In [2]:
ICIPpath = '/home/jguerry/workspace/ICIPmodels/'
mono_deploy = ICIPpath + 'mono_deploy.prototxt'
fuse_deploy = ICIPpath + 'fuse_deploy.prototxt'
weights_path_0='/home/jguerry/workspace/ICIPmodels/rgb_100_weights.caffemodel'
weights_path_1='/home/jguerry/workspace/ICIPmodels/dha_weights.caffemodel'
#weights_path_1='/home/jguerry/workspace/ICIPmodels/cube_weights.caffemodel'
weights_path_fuse='/home/jguerry/workspace/ICIPmodels/fuse_weights.caffemodel'

In [3]:
def transfer(new_net, net, suffix=''):
    # from fcn.berkeleyvision.org
    for p in net.params:
        #print "Working on", p, ':'
        p_new = p + suffix
        if p_new not in new_net.params:
            print 'dropping', p
            continue
        for i in range(len(net.params[p])):
            if i > (len(new_net.params[p_new]) - 1):
                print 'dropping', p, i
                break
            if net.params[p][i].data.shape != new_net.params[p_new][i].data.shape:
                print 'coercing', p, i, 'from', net.params[p][i].data.shape, 'to', new_net.params[p_new][i].data.shape
            else:
                print 'copying', p, ' -> ', p_new, i
            new_net.params[p_new][i].data.flat = net.params[p][i].data.flat

In [4]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [5]:
monoNet = caffe.Net(mono_deploy, caffe.TEST)
monoBlobs = [(k, v.data.shape) for k, v in monoNet.blobs.items()]
monoLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in monoNet.params.items() if k != 'upscore37']

In [6]:
fuseNet = caffe.Net(fuse_deploy, caffe.TEST)
fuseBlobs = [(k, v.data.shape) for k, v in fuseNet.blobs.items()]
fuseLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in fuseNet.params.items() if k not in ['upscore37_data0','upscore37_data1','upscore_sum', 'upscore_concat','upscore_rc_corrected']]

In [7]:
monoNet.copy_from(weights_path_0,)
transfer(fuseNet,monoNet,'_data0')
monoNet.copy_from(weights_path_1)
transfer(fuseNet,monoNet,'_data1')

copying conv1_1  ->  conv1_1_data0 0
copying conv1_1  ->  conv1_1_data0 1
copying conv1_2  ->  conv1_2_data0 0
copying conv1_2  ->  conv1_2_data0 1
copying conv2_1  ->  conv2_1_data0 0
copying conv2_1  ->  conv2_1_data0 1
copying conv2_2  ->  conv2_2_data0 0
copying conv2_2  ->  conv2_2_data0 1
copying conv3_1  ->  conv3_1_data0 0
copying conv3_1  ->  conv3_1_data0 1
copying conv3_2  ->  conv3_2_data0 0
copying conv3_2  ->  conv3_2_data0 1
copying conv3_3  ->  conv3_3_data0 0
copying conv3_3  ->  conv3_3_data0 1
copying conv4_1  ->  conv4_1_data0 0
copying conv4_1  ->  conv4_1_data0 1
copying conv4_2  ->  conv4_2_data0 0
copying conv4_2  ->  conv4_2_data0 1
copying conv4_3  ->  conv4_3_data0 0
copying conv4_3  ->  conv4_3_data0 1
copying conv5_1  ->  conv5_1_data0 0
copying conv5_1  ->  conv5_1_data0 1
copying conv5_2  ->  conv5_2_data0 0
copying conv5_2  ->  conv5_2_data0 1
copying conv5_3  ->  conv5_3_data0 0
copying conv5_3  ->  conv5_3_data0 1
copying fc6  ->  fc6_data0 0
copying f

In [8]:
#### SAVING
print "Saving output model to %s " % weights_path_fuse
fuseNet.save(weights_path_fuse)

Saving output model to /home/jguerry/workspace/ICIPmodels/fuse_weights.caffemodel 
